In [1]:
import sys
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.tri import Triangulation
from scipy.interpolate import griddata

## `get_dir()`

In [2]:
def get_dir(name):
    """
    Gera o caminho completo para salvar o arquivo e garante que o diretório exista.
    
    Parâmetros:
        name (str): Nome do arquivo com o caminho relativo a partir do diretório base.

    Retorna:
        str: Caminho completo para o arquivo, tratado para espaços e barras.
    """
    base_dir = os.getcwd()  # Obtém o diretório atual
    filepath = os.path.normpath(os.path.join(base_dir, name))  # Normaliza o caminho completo
    directory = os.path.dirname(filepath)  # Obtém o diretório onde o arquivo será salvo

    # Garante que o diretório exista
    os.makedirs(directory, exist_ok=True)
    
    # Retorna o caminho normalizado
    return filepath

## `conn_py()`

In [3]:
def conn(mesh_data):
    # Matriz de conectividade, ajustada para indexação Python (iniciando em zero)
    return np.array([element['conn_list'] for element in mesh_data['conn_data']]) - 1

## `get_nodes()`

In [4]:
def get_nodes_array(mesh_data):
    nodes_list = [node['global_coord'] for node in mesh_data['nodes_data']]
    nodes_x = np.array([node[0] for node in nodes_list])
    nodes_y = np.array([node[1] for node in nodes_list])
    nodes = np.column_stack((nodes_x, nodes_y))
    return nodes_x, nodes_y, nodes

## `mesh_quad()`

In [5]:
def mesh_quad(mesh_data):
    nodes_x, nodes_y, _ = get_nodes_array(mesh_data)
    conn = conn(mesh_data) + 1
    conn_quad = np.array([cell[:4] for cell in conn])
    for cell in conn_quad:
        # Extrair coordenadas x e y dos quatro nós do quadrilátero
        x = [nodes_x[node - 1] for node in cell]
        y = [nodes_y[node - 1] for node in cell]

        for i in range(4):
            # Conectar nó i com nó (i+1) % 4 para fechar o quadrilátero
            x_edge = [x[i], x[(i + 1) % 4]]
            y_edge = [y[i], y[(i + 1) % 4]]
            plt.plot(x_edge, y_edge, color='gray', zorder=1)

## `plot_numbers()`

In [6]:
def plot_numbers(FINITE_ELEMENT, mesh_data):

    type, _ = FINITE_ELEMENT

    # Extraindo as coordenadas globais dos nós (x, y)
    xg = [node['xg'][0] for node in mesh_data['nodes'].values()]
    yg = [node['xg'][1] for node in mesh_data['nodes'].values()]

    # Extraindo a matriz de conectividade
    conn = [cell['conn'] for cell in mesh_data['cell'].values()]
    conn_py = [[vertex - 1 for vertex in cell] for cell in conn]

    if type == "Triangle":
        for i, (x, y) in enumerate(zip(xg, yg)):
            plt.scatter(x, y, color='white', edgecolor='black', s=60, zorder=1)  
            plt.text(x, y, str(i+1), color='red', fontsize=4, ha='center', va='center')

        # Adicionando os números dos elementos no centro
        for i, cell in enumerate(conn_py):
            # Calculando o centroide do elemento
            x_c = np.mean([xg[vertex] for vertex in cell])
            y_c = np.mean([yg[vertex] for vertex in cell])
            
            # Adicionando o número do elemento
            plt.text(x_c, y_c, str(i+1), color='black', fontsize=4, ha='center', va='center', zorder=1)

# `structured_data()`

In [ ]:
def structured_data(mesh_data):
    # Extraindo as coordenadas globais dos nós (x, y)
    try:
        xg = [node['xg'][0] for node in mesh_data['nodes'].values()]
        yg = [node['xg'][1] for node in mesh_data['nodes'].values()]
    except KeyError as e:
        raise ValueError(f"Erro ao acessar as coordenadas globais: {e}")

    # Extraindo a matriz de conectividade
    try:
        conn = [cell['conn'] for cell in mesh_data['cell'].values()]
        # Ajusta índice para 0-based
        conn_py = [[node - 1 for node in nodes[:3]] for nodes in conn]  
    except KeyError as e:
        raise ValueError(f"Erro ao acessar a conectividade: {e}")
    
    return xg, yg, conn_py

## `plot_mesh()`

In [7]:
def plot_mesh(FINITE_ELEMENT, mesh_data, model, Numbering=False):
    type, order = FINITE_ELEMENT
    name = f"pre_processing/pictures/{model}_meshed_domain_{type}{order}.svg"
    filepath = get_dir(name)
    
    # Ajustar o tamanho dos pontos dependendo da ordem do elemento
    size_dot = 5 if order == 1 else 2

    # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
    xg, yg, conn_py = structured_data(mesh_data)

    plt.figure(figsize=(8, 6))
    if type == "Triangle":
        plt.triplot(xg, yg, conn_py, color='gray')  
        if Numbering:            
            # Adicionando os números dos nós
            for i, (x, y) in enumerate(zip(xg, yg)):
                plt.scatter(x, y, color='white', edgecolor='black', s=60, zorder=1)  
                plt.text(x, y, str(i+1),
                          color='red', fontsize=4, ha='center', va='center')

            # Adicionando os números dos elementos no centro
            for i, cell in enumerate(conn_py):
                x_c = np.mean([xg[vertex] for vertex in cell])
                y_c = np.mean([yg[vertex] for vertex in cell])
                plt.text(x_c, y_c, str(i+1),
                          color='black', fontsize=4, ha='center', va='center', zorder=1)

    elif type == "Quadrangle":
        mesh_quad(mesh_data)

    if not Numbering:
        plt.scatter(xg, yg, color='red', edgecolor='black', s=size_dot, zorder=3)   
        
    # Ajustando rótulos e layout
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()
    
    # Salvando o arquivo no formato SVG
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

## `approximate_not_structured_quadrangles()`

In [8]:
def approximate_not_structured_quadrangles(element_type, mesh_data, u, model, FlagGrid=True):
        
    # Descompactar o tipo e a ordem dos elementos
    type, order = element_type
    name = f"pos_processing/pictures/{model}_domain_solution_{type}{order}_not_structured.svg"
    filepath = get_dir(name)
    
    # Obter coordenadas dos nós e conectividade da malha
    nodes_x, nodes_y, nodes = get_nodes_array(mesh_data)
    conn = np.array([element['conn_list'] for element in mesh_data['conn_data']])
    
    # Gerar uma grade regular para interpolação
    grid_x, grid_y = np.mgrid[
        np.min(nodes[:, 0]):np.max(nodes[:, 0]):200j,
        np.min(nodes[:, 1]):np.max(nodes[:, 1]):200j,
    ]
    
    # Interpolação com base no tipo de elemento
    if type == "Quadrangle":
        if order == 1:
            # Interpolação linear para elementos Q1
            grid_u = griddata(nodes, u, (grid_x, grid_y), method='linear')
        elif order == 2:
            # Interpolação cúbica para elementos Q2
            grid_u = griddata(nodes, u, (grid_x, grid_y), method='cubic')
        else:
            raise ValueError("Apenas ordens 1 (Q1) e 2 (Q2) são suportadas para elementos quadrilaterais.")
    else:
        raise ValueError("Tipo de elemento não suportado. Use 'Quadrangle'.")
    
    # Plotar a solução interpolada
    plt.figure(figsize=(8, 6))
    contour = plt.contourf(grid_x, grid_y, grid_u, levels=100, cmap='viridis')
    plt.colorbar(contour)
    
    # Plotar a malha se FlagGrid=True
    if FlagGrid:
        conn_quad = [cell[:4] for cell in conn]
        for element in conn_quad:
            element = element - 1  # Ajustar para indexação Python
            polygon = nodes[element, :]  # Coordenadas dos nós do elemento
            plt.plot(
                np.append(polygon[:, 0], polygon[0, 0]),  # Fechar o polígono
                np.append(polygon[:, 1], polygon[0, 1]),
                color='gray', linewidth=0.5
            )
    
    plt.xlabel('$x$')
    plt.ylabel('$y$')
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

## `approximate_fem_griddata()` 

In [9]:
def fem_griddata(FINITE_ELEMENT, mesh_data, u_dict, type='real'):
    """
    Plota a solução aproximada u(x,y) sobre o domínio retangular [0,1]^2
    com suporte para diferentes tipos de elementos e ordens, incluindo Q2 com 9 nós.
    
    Parâmetros:
    - mesh_data: dicionário contendo 'nodes' e 'conn' (conectividade) do domínio.
    - u: array com os valores da função aproximada em cada nó, com shape (N,).
    - ElementType: tupla (string, ordem) que indica o tipo e a ordem dos elementos.
    """
    # Conversão do dicionário de soluções para lista
    if type == 'real':
        u = np.real(list(u_dict.values()))
    elif type == 'imag':
        u = np.imag(list(u_dict.values()))
    elif type == 'abs':
        u = np.abs(list(u_dict.values()))
    else:
        raise ValueError("Tipo de solução inválido. Use 'real', 'imag' ou 'abs'.")
    
    # Gerar uma grade regular para interpolação
    type, order = FINITE_ELEMENT
    
    # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
    xg, yg, _ = structured_data(mesh_data)
    nodes = np.column_stack((xg, yg))
    
    # Gerar uma grade regular para interpolação
    grid_x, grid_y = np.mgrid[0:1:500j, 0:1:500j]
    
    # Interpolação cúbica para elementos triangulares
    grid_u = griddata(nodes, u, (grid_x, grid_y), method='cubic')
    
    # Plotar a solução interpolada
    plt.figure(figsize=(8, 6))
    plt.colorbar(plt.contourf(grid_x, grid_y, grid_u, levels=100, cmap='viridis'))
        
    plt.xlabel('x')
    plt.ylabel('y')
    plt.tight_layout()
    
    # Salvar o arquivo no formato SVG
    filepath = get_dir(f"pos_processing/pictures/fem_solution_griddata_{type}{order}.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

## `fem_solution()`

In [10]:
def fem_solution(FINITE_ELEMENT, mesh_data, u_dict, type='real'):
    # Conversão do dicionário de soluções para lista
    if type == 'real':
        u = np.real(list(u_dict.values()))
    elif type == 'imag':
        u = np.imag(list(u_dict.values()))
    elif type == 'abs':
        u = np.abs(list(u_dict.values()))
    else:
        raise ValueError("Tipo de solução inválido. Use 'real', 'imag' ou 'abs'.")

    # Extração de informações sobre o elemento finito
    type, order = FINITE_ELEMENT

    # Extraindo as coordenadas globais dos nós (x, y) e a matriz de conectividade
    xg, yg, conn = structured_data(mesh_data)

    # Verificação do tipo e ordem do elemento
    if type == "Triangle":
        triangulation = Triangulation(xg, yg, conn)
    else:
        raise ValueError("Apenas elementos triangulares ('Triangle') são suportados.")

    # Plot da solução
    plt.figure(figsize=(8, 6))
    plt.tricontourf(triangulation, u, cmap='viridis')
    plt.colorbar(label=r'$u(x, y)$')
    plt.triplot(triangulation, color='gray', alpha=0.5)
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.tight_layout()

    plt.show()
    # # Salvando o gráfico
    # filepath = get_dir(f"pos_processing/pictures/fem_solution_{type}{order}.svg")
    # plt.savefig(filepath, format="svg")
    # plt.close()
    # print(f"Arquivo salvo em: {filepath}")

# `L_analytical_solution()`

In [12]:
def L_analytical_solution(exact_potential, Npts=100):
    name = f"pos_processing/pictures/L_analytical_solution.svg"
    filepath = get_dir(name)
    # Determinar limites do domínio
    vertices = [(0, 0, 0), (0, -1, 0), (1, -1, 0), (1, 1, 0), (-1, 1, 0), (-1, 0, 0)]
    x_coords = [v[0] for v in vertices]
    y_coords = [v[1] for v in vertices]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)

    # Criar uma grade de pontos (x, y) no domínio
    x = np.linspace(x_min, x_max, Npts)
    y = np.linspace(y_min, y_max, Npts)
    X, Y = np.meshgrid(x, y)

    # Máscara para o domínio em forma de L
    mask = np.zeros_like(X, dtype=bool)
    for i in range(Npts):
        for j in range(Npts):
            px, py = X[i, j], Y[i, j]
            # Verificar se o ponto (px, py) está dentro do domínio L
            if (px >= 0 or py >= 0):  # Parte direita ou parte superior
                mask[i, j] = True

    # Calcular o potencial analítico no domínio válido
    potential = np.full_like(X, np.nan, dtype=float)  # Preencher com NaN para fora do domínio
    for i in range(Npts):
        for j in range(Npts):
            if mask[i, j]:  # Apenas pontos dentro do domínio L
                potential[i, j] = exact_potential(X[i, j], Y[i, j])

    # Plotar o potencial
    plt.figure(figsize=(8, 6))
    contour = plt.contourf(X, Y, potential, levels=50, cmap='viridis')
    plt.colorbar(contour, label=r"$u(x, y)$")
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.axis("equal")
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `plot_convergence()`

In [13]:
def plot_convergence_with_slope(mesh_sizes, error_results, model, name, points_coord=False):
    filename = f"pos_processing/pictures/{model}_convergence_{name}.svg"
    filepath = get_dir(filename)
    plt.figure(figsize=(10, 8))
    
    for element_type, error_list in error_results.items():
        # Extraindo os valores escalares dos arrays
        error_list = [float(val) for val in error_list]
        
        # Plotando os pontos
        plt.loglog(mesh_sizes, error_list, marker='o', linestyle='-', label=f'{element_type[0]} Order {element_type[1]}')
        
        # Calculando as inclinações
        slopes = []
        for i in range(1, len(mesh_sizes)):
            delta_x = np.log10(mesh_sizes[i]) - np.log10(mesh_sizes[i-1])
            delta_y = np.log10(error_list[i]) - np.log10(error_list[i-1])
            slope = delta_y / delta_x
            slopes.append(slope)
            
            # Adicionando o valor da inclinação no meio do segmento
            mid_x = np.sqrt(mesh_sizes[i] * mesh_sizes[i-1])
            mid_y = np.sqrt(error_list[i] * error_list[i-1])
            plt.text(mid_x, mid_y, f"{slope:.4f}", fontsize=8, ha='center', va='bottom', color='blue')

        # Adicionando os valores (x, y) ao lado de cada ponto
        if points_coord:
            for x, y in zip(mesh_sizes, error_list):
                plt.text(float(x), float(y), f"({x:.4e},\n{y:.4e})", fontsize=7, ha='right', va='bottom')

    plt.xlabel(r'Element size $h$')
    plt.ylabel(f'{name} norm of the error')
    plt.legend(title="Element Types")
    plt.grid(which='both', linestyle='--', linewidth=0.5)  
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")


In [14]:
def plot_convergence(mesh_sizes, error_results, model, name):
    filename = f"pos_processing/pictures/{model}_convergence_{name}.svg"
    filepath = get_dir(filename)
    plt.figure(figsize=(10, 8))
    
    for element_type, error_list in error_results.items():
        plt.loglog(mesh_sizes, error_list, marker='o', linestyle='-', label=f'{element_type[0]} Order {element_type[1]}')

    plt.xlabel(r'Element size $h$')
    plt.ylabel(f'{name} norm of the error')
    plt.legend(title="Element Types")
    plt.grid(which='both', linestyle='--', linewidth=0.5)  
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `plot_capacitance_convergence()`

In [15]:
def capacitance_convergence(mesh_sizes, capacitance):
    filename = f"pos_processing/pictures/coaxial_capacitance_convergence.svg"
    filepath = get_dir(filename)
    plt.figure(figsize=(10, 8))
    
    for element_type, values in capacitance.items():
        type, order = element_type
        plt.semilogx(mesh_sizes, np.array(values)*1E12, marker='o', linestyle='-', label=f'{type} Order {order}')
        
        # Adiciona os valores próximos aos pontos
        for h, value in zip(mesh_sizes, values):
            plt.text(h, value * 1E12 * 1.001, f'{value * 1E12:.4f}', fontsize=8, ha='center')

    plt.xlabel(r'Element size, $h$')
    plt.ylabel(r'Capacitance ($pF$)')
    plt.legend(title="Element Types")
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `magnetic_field_distribution()`

In [16]:
def magnetic_field_distribution(mesh_data, B_domain, element_type):
    """
    Plota a distribuição do valor absoluto do campo magnético |B| no domínio.

    Parameters:
    mesh_data: dict
        Dados da malha, incluindo coordenadas dos nós e conectividade.
    B_domain: list of arrays
        Lista contendo os valores de B nos pontos de Gauss de cada elemento.
    element_type: tuple
        Tipo e ordem do elemento no formato (type, order). 
        Exemplo: ('triangle', 1) para elementos P1 ou ('triangle', 2) para elementos P2.
    """
    # Desestrutura o tipo e a ordem do elemento
    type, order = element_type
    assert type == 'Triangle', "Somente elementos triangulares são suportados"
    assert order in [1, 2, 3], "Apenas elementos P1 (linear) e P2 (quadrático) são suportados"

    # Nome do arquivo de saída
    filename = f"pos_processing/pictures/b_field_{type}{order}.svg"
    filepath = get_dir(filename)

    # Obtém as coordenadas dos nós e os dados de conectividade
    nodes_x, nodes_y, nodes = get_nodes_array(mesh_data)
    conn = [element['conn_list'] for element in mesh_data['conn_data']]

    # Ajusta a conectividade para índices baseados em 0
    conn = np.array(conn) - 1

    # Inicializa uma lista para armazenar o valor médio de |B| em cada elemento
    B_magnitude_elements = []

    # Calcula a magnitude média de |B| para cada elemento
    for e, B_element in enumerate(B_domain):
        # Calcula a magnitude de B em cada ponto de Gauss do elemento
        B_magnitudes = [np.linalg.norm(B) for B in B_element]
        
        # Calcula o valor médio de |B| para o elemento
        B_magnitude_avg = np.mean(B_magnitudes)
        B_magnitude_elements.append(B_magnitude_avg)

    # Inicializa os vetores para armazenar os valores em cada nó
    B_magnitude_per_node = np.zeros(len(nodes_x))
    element_count_per_node = np.zeros(len(nodes_x))

    # Distribui o valor médio de |B| dos elementos para os nós
    for e, element in enumerate(conn):
        # Considera os nós do elemento, variando conforme P1 ou P2
        for node in element:
            B_magnitude_per_node[node] += B_magnitude_elements[e]
            element_count_per_node[node] += 1

    # Calcula o valor médio de |B| por nó
    B_magnitude_per_node /= element_count_per_node

    # Triangulação para visualização (P1 usa nós diretos; P2 precisa de adaptação)
    if order == 1:
        triangulation = Triangulation(nodes_x, nodes_y, conn)
    elif order == 2 or order == 3:
        # Elementos P2 incluem nós intermediários, mas ainda podem usar Triangulation
        triangulation = Triangulation(nodes_x, nodes_y, conn[:, :3])
    
    # Plotagem da distribuição
    plt.figure(figsize=(8, 6))
    plt.tricontourf(triangulation, B_magnitude_per_node, levels=100, cmap='gray_r')
    plt.colorbar(label=r'|$\mathbf{B}$| (T)')
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.axis('equal')
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()

    print(f"Arquivo salvo em: {filepath}")

# `magnetic_field_distribution_griddata()`

In [17]:
def magnetic_field_distribution_griddata(mesh_data, B_domain, element_type):
    """
    Plota a distribuição do valor absoluto do campo magnético |B| no domínio.

    Parameters:
    mesh_data: dict
        Dados da malha, incluindo coordenadas dos nós e conectividade.
    B_domain: list of arrays
        Lista contendo os valores de B nos pontos de Gauss de cada elemento.
    element_type: tuple
        Tipo e ordem do elemento no formato (type, order).
    cmap: str, optional
        Mapa de cores para a visualização (default: 'gray_r').
    resolution: int, optional
        Resolução da grade para interpolação (default: 500).
    figsize: tuple, optional
        Tamanho da figura em polegadas (default: (8, 6)).
    """
    # Desestrutura o tipo e a ordem do elemento
    resolution=500
    cmap='gray_r'
    figsize=(8, 6)
    elem_type, order = element_type
    assert elem_type == 'Triangle', "Somente elementos triangulares são suportados"
    assert order in [1, 2, 3], "Apenas elementos P1 (linear) e P2 (quadrático) são suportados"

    # Nome do arquivo de saída
    filename = f"pos_processing/pictures/b_field_griddata_{elem_type}{order}.svg"
    filepath = get_dir(filename)

    # Obtém as coordenadas dos nós e os dados de conectividade
    nodes_x, nodes_y, nodes = get_nodes_array(mesh_data)
    conn = [element['conn_list'] for element in mesh_data['conn_data']]

    # Ajusta a conectividade para índices baseados em 0
    conn = np.array(conn) - 1

    # Inicializa uma lista para armazenar o valor médio de |B| em cada elemento
    B_magnitude_elements = []

    # Calcula a magnitude média de |B| para cada elemento
    for e, B_element in enumerate(B_domain):
        # Calcula a magnitude de B em cada ponto de Gauss do elemento
        B_magnitudes = [np.linalg.norm(B) for B in B_element]
        
        # Calcula o valor médio de |B| para o elemento
        B_magnitude_avg = np.mean(B_magnitudes)
        B_magnitude_elements.append(B_magnitude_avg)

    # Calcula as coordenadas médias dos elementos (centroides)
    centroids_x = []
    centroids_y = []
    for element in conn:
        element_coords_x = nodes_x[element]
        element_coords_y = nodes_y[element]
        centroids_x.append(np.mean(element_coords_x))
        centroids_y.append(np.mean(element_coords_y))

    # Interpolação com griddata
    grid_x, grid_y = np.linspace(min(nodes_x), max(nodes_x), resolution), np.linspace(min(nodes_y), max(nodes_y), resolution)
    grid_x, grid_y = np.meshgrid(grid_x, grid_y)

    # Interpolação dos valores médios |B| nos centroides
    grid_B = griddata((centroids_x, centroids_y), B_magnitude_elements, (grid_x, grid_y), method='linear')

    # Plotagem da distribuição
    plt.figure(figsize=figsize)
    plt.contourf(grid_x, grid_y, grid_B, levels=100, cmap=cmap)
    plt.colorbar(label=r'|$\mathbf{B}$| (T)')
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.axis('equal')
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()

    print(f"Arquivo salvo em: {filepath}")


# `magnetic_field_distribution_imshow()`

In [18]:
def magnetic_field_distribution_imshow(mesh_data, B_domain, element_type):
    """
    Plota a distribuição do valor absoluto do campo magnético |B| no domínio usando imshow.

    Parameters:
    mesh_data: dict
        Dados da malha, incluindo coordenadas dos nós e conectividade.
    B_domain: list of arrays
        Lista contendo os valores de B nos pontos de Gauss de cada elemento.
    element_type: tuple
        Tipo e ordem do elemento no formato (type, order).
    cmap: str, optional
        Mapa de cores para a visualização (default: 'viridis').
    resolution: int, optional
        Resolução da grade para interpolação (default: 500).
    figsize: tuple, optional
        Tamanho da figura em polegadas (default: (8, 6)).
    """
    # Desestrutura o tipo e a ordem do elemento
    resolution=500
    cmap='gray_r'
    figsize=(8, 6)
    elem_type, order = element_type
    assert elem_type == 'Triangle', "Somente elementos triangulares são suportados"
    assert order in [1, 2, 3], "Apenas elementos P1 (linear) e P2 (quadrático) são suportados"

    # Nome do arquivo de saída
    filename = f"pos_processing/pictures/b_field_imshow_{elem_type}{order}.svg"
    filepath = get_dir(filename)

    # Obtém as coordenadas dos nós e os dados de conectividade
    nodes_x, nodes_y, nodes = get_nodes_array(mesh_data)
    conn = [element['conn_list'] for element in mesh_data['conn_data']]

    # Ajusta a conectividade para índices baseados em 0
    conn = np.array(conn) - 1

    # Inicializa uma lista para armazenar o valor médio de |B| em cada elemento
    B_magnitude_elements = []

    # Calcula a magnitude média de |B| para cada elemento
    for e, B_element in enumerate(B_domain):
        # Calcula a magnitude de B em cada ponto de Gauss do elemento
        B_magnitudes = [np.linalg.norm(B) for B in B_element]
        
        # Calcula o valor médio de |B| para o elemento
        B_magnitude_avg = np.mean(B_magnitudes)
        B_magnitude_elements.append(B_magnitude_avg)

    # Calcula as coordenadas médias dos elementos (centroides)
    centroids_x = []
    centroids_y = []
    for element in conn:
        element_coords_x = nodes_x[element]
        element_coords_y = nodes_y[element]
        centroids_x.append(np.mean(element_coords_x))
        centroids_y.append(np.mean(element_coords_y))

    # Interpolação com griddata
    grid_x, grid_y = np.linspace(min(nodes_x), max(nodes_x), resolution), np.linspace(min(nodes_y), max(nodes_y), resolution)
    grid_x, grid_y = np.meshgrid(grid_x, grid_y)

    # Interpolação dos valores médios |B| nos centroides
    grid_B = griddata((centroids_x, centroids_y), B_magnitude_elements, (grid_x, grid_y), method='linear')

    # Plotagem da distribuição usando imshow
    plt.figure(figsize=figsize)
    plt.imshow(grid_B.T, extent=(min(nodes_x), max(nodes_x), min(nodes_y), max(nodes_y)), 
               origin='lower', cmap=cmap, aspect='auto')
    plt.colorbar(label=r'|$\mathbf{B}$| (T)')
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.axis('equal')
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()

    print(f"Arquivo salvo em: {filepath}")


# `magnetic_flux_lines()`

In [19]:
def magnetic_flux_lines(mesh_data, B_domain, element_type):
    """
    Plota as linhas de fluxo magnético no domínio com base na malha e no campo magnético.

    Parameters:
    mesh_data: dict
        Dados da malha, incluindo coordenadas dos nós e conectividade.
    B_domain: list of arrays
        Lista contendo os valores de B nos pontos de Gauss de cada elemento.
    element_type: tuple
        Tipo e ordem do elemento no formato (type, order).
        Exemplo: ('triangle', 1) para elementos P1 ou ('triangle', 2) para elementos P2.
    n_lines: int, opcional
        Número de linhas de fluxo a serem desenhadas (default: 20).
    """
    # Desestrutura o tipo e a ordem do elemento
    type, order = element_type
    assert type == 'Triangle', "Somente elementos triangulares são suportados"
    assert order in [1, 2, 3], "Apenas elementos P1 (linear) e P2 (quadrático) são suportados"

    # Nome do arquivo de saída
    filename = f"pos_processing/pictures/flux_lines_{type}{order}.svg"
    filepath = get_dir(filename)

    # Obtém as coordenadas dos nós e os dados de conectividade
    nodes_x, nodes_y, nodes = get_nodes_array(mesh_data)
    conn = [element['conn_list'] for element in mesh_data['conn_data']]

    # Ajusta a conectividade para índices baseados em 0
    conn = np.array(conn) - 1

    # Inicializa listas para armazenar as coordenadas dos pontos de Gauss e os vetores B
    gauss_points = []
    B_vectors = []

    for element, B_element in zip(conn, B_domain):
        # Obtém as coordenadas dos nós do elemento
        element_coords = np.array([[nodes_x[node], nodes_y[node]] for node in element])
        # Adiciona os pontos de Gauss e os valores de B associados
        for i, B in enumerate(B_element):
            gauss_points.append(np.mean(element_coords, axis=0))  # Usa o centroide como aproximação
            B_vectors.append(B)

    gauss_points = np.array(gauss_points)
    B_vectors = np.array(B_vectors)

    # Cria um grid uniforme para interpolar o campo vetorial
    x_min, x_max = nodes_x.min(), nodes_x.max()
    y_min, y_max = nodes_y.min(), nodes_y.max()
    grid_x, grid_y = np.linspace(x_min, x_max, 200), np.linspace(y_min, y_max, 200)
    grid_x, grid_y = np.meshgrid(grid_x, grid_y)

    # Interpola os componentes do campo magnético Bx e By
    Bx = griddata(gauss_points, B_vectors[:, 0], (grid_x, grid_y), method='cubic', fill_value=0)
    By = griddata(gauss_points, B_vectors[:, 1], (grid_x, grid_y), method='cubic', fill_value=0)

    # Plotagem das linhas de fluxo
    plt.figure(figsize=(8, 6))
    plt.streamplot(grid_x, grid_y, Bx, By, color=np.sqrt(Bx**2 + By**2), linewidth=1, cmap='viridis', density=1.5)
    plt.colorbar(label=r'|$\mathbf{B}$| (T)')
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')
    plt.axis('equal')
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()

    print(f"Linhas de fluxo magnético salvas em: {filepath}")


# `radial_profile()`

In [20]:
def radial_profile(mesh_data, B_domain, element_type):
    """
    Plota o perfil do módulo do campo magnético |B| ao longo da distância radial
    a partir do centro do domínio.

    Parameters:
    mesh_data: dict
        Dados da malha, incluindo coordenadas dos nós e conectividade.
    B_domain: list of arrays
        Lista contendo os valores de B nos pontos de Gauss de cada elemento.
    element_type: tuple
        Tipo e ordem do elemento no formato (type, order). 
        Exemplo: ('triangle', 1) para elementos P1 ou ('triangle', 2) para elementos P2.
    """
    # Desestrutura o tipo e a ordem do elemento
    element_geom, order = element_type
    assert element_geom == 'Triangle', "Somente elementos triangulares são suportados"
    assert order in [1, 2, 3], "Apenas elementos P1 (linear) e P2 (quadrático) são suportados"

    # Nome do arquivo de saída
    filename = f"pos_processing/pictures/radial_b_profile_{element_geom}{order}.svg"
    filepath = get_dir(filename)

    # Obtém as coordenadas dos nós e os dados de conectividade
    nodes_x, nodes_y, nodes = get_nodes_array(mesh_data)
    conn = [element['conn_list'] for element in mesh_data['conn_data']]
    conn = np.array(conn) - 1  # Ajusta a conectividade para índices baseados em 0

    # Inicializa listas para armazenar as magnitudes de |B| e as distâncias radiais
    radial_distances = []
    B_magnitudes = []

    # Calcula o módulo de |B| em cada ponto de Gauss dos elementos e sua distância radial
    for e, B_element in enumerate(B_domain):
        # Obtém a conectividade do elemento
        element_nodes = conn[e]

        # Para P1, calcula o centroide diretamente; para P2, considera apenas os nós vértices
        if order == 1:
            element_coords = nodes[element_nodes]
        elif order == 2 or order == 3:
            element_coords = nodes[element_nodes[:3]]  # Apenas os 3 primeiros nós (vértices)

        # Calcula o centroide do elemento
        centroid = np.mean(element_coords, axis=0)  # Centroide (x, y) do elemento

        # Calcula a distância radial do centroide
        r = np.linalg.norm(centroid)
        radial_distances.append(r)

        # Calcula a magnitude média de |B| para o elemento
        B_magnitudes_element = [np.linalg.norm(B) for B in B_element]
        B_magnitude_avg = np.mean(B_magnitudes_element)
        B_magnitudes.append(B_magnitude_avg)

    # Ordena os valores em função da distância radial
    radial_distances = np.array(radial_distances)
    B_magnitudes = np.array(B_magnitudes)
    sorted_indices = np.argsort(radial_distances)
    radial_distances_sorted = radial_distances[sorted_indices]
    B_magnitudes_sorted = B_magnitudes[sorted_indices]

    # Plotagem do gráfico
    plt.figure(figsize=(8, 6))
    plt.plot(radial_distances_sorted, B_magnitudes_sorted, label=r'|$\mathbf{B}$|')
    plt.xlabel('Radial distance (m)')
    plt.ylabel(r'|$\mathbf{B}$| (T)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `lateral_profile()`

In [21]:
def lateral_profile(mesh_data, B_domain, element_type, y=0):
    """
    Plota o perfil do módulo do campo magnético |B| ao longo de x no centro do domínio.

    Parameters:
    mesh_data: dict
        Dados da malha, incluindo coordenadas dos nós e conectividade.
    B_domain: list of arrays
        Lista contendo os valores de B nos pontos de Gauss de cada elemento.
    element_type: tuple
        Tipo e ordem do elemento no formato (type, order). 
        Exemplo: ('triangle', 1) para elementos P1 ou ('triangle', 2) para elementos P2.
    """
    # Desestrutura o tipo e a ordem do elemento
    element_geom, order = element_type
    assert element_geom == 'Triangle', "Somente elementos triangulares são suportados"
    assert order in [1, 2, 3], "Apenas elementos P1 (linear) e P2 (quadrático) são suportados"

    # Nome do arquivo de saída
    filename = f"pos_processing/pictures/lateral_b_profile_{element_geom}{order}.svg"
    filepath = get_dir(filename)

    # Obtém as coordenadas dos nós e os dados de conectividade
    nodes_x, nodes_y, nodes = get_nodes_array(mesh_data)
    conn = [element['conn_list'] for element in mesh_data['conn_data']]
    conn = np.array(conn) - 1  # Ajusta a conectividade para índices baseados em 0

    # Inicializa listas para armazenar |B| e as coordenadas x
    x_coords = []
    B_magnitudes = []

    # Calcula o módulo de |B| e filtra os elementos no centro do domínio (y ≈ 0.5)
    for e, B_element in enumerate(B_domain):
        # Obtém a conectividade do elemento
        element_nodes = conn[e]

        # Para P1, usa o centroide; para P2, considera apenas os nós vértices
        if order == 1:
            element_coords = nodes[element_nodes]
        elif order == 2 or order == 3:
            element_coords = nodes[element_nodes[:3]]

        # Calcula o centroide do elemento
        centroid = np.mean(element_coords, axis=0)  # Centroide (x, y)

        # Filtra elementos próximos de y
        if np.isclose(centroid[1], y, atol=0.05):  # Tolerância de 0.05
            x_coords.append(centroid[0])  # Coordenada x do centroide
            # Calcula a magnitude média de |B| para o elemento
            B_magnitudes_element = [np.linalg.norm(B) for B in B_element]
            B_magnitude_avg = np.mean(B_magnitudes_element)
            B_magnitudes.append(B_magnitude_avg)

    # Ordena os valores em função de x
    x_coords = np.array(x_coords)
    B_magnitudes = np.array(B_magnitudes)
    sorted_indices = np.argsort(x_coords)
    x_coords_sorted = x_coords[sorted_indices]
    B_magnitudes_sorted = B_magnitudes[sorted_indices]

    # Plotagem do gráfico
    plt.figure(figsize=(8, 6))
    plt.semilogy(x_coords_sorted, B_magnitudes_sorted, label=r'|$\mathbf{B}$|')
    plt.xlabel('x-coordinate (m)')
    plt.ylabel(r'|$\mathbf{B}$| (T)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

# `diagonal_profile()`

In [22]:
def diagonal_profile(mesh_data, B_domain, element_type):
    """
    Plota o perfil do módulo do campo magnético |B| ao longo da diagonal do domínio (x = y).

    Parameters:
    mesh_data: dict
        Dados da malha, incluindo coordenadas dos nós e conectividade.
    B_domain: list of arrays
        Lista contendo os valores de B nos pontos de Gauss de cada elemento.
    element_type: tuple
        Tipo e ordem do elemento no formato (type, order). 
        Exemplo: ('triangle', 1) para elementos P1 ou ('triangle', 2) para elementos P2.
    """
    # Desestrutura o tipo e a ordem do elemento
    element_geom, order = element_type
    assert element_geom == 'Triangle', "Somente elementos triangulares são suportados"
    assert order in [1, 2, 3], "Apenas elementos P1 (linear) e P2 (quadrático) são suportados"

    # Nome do arquivo de saída
    filename = f"pos_processing/pictures/diagonal_b_profile_{element_geom}{order}.svg"
    filepath = get_dir(filename)

    # Obtém as coordenadas dos nós e os dados de conectividade
    nodes_x, nodes_y, nodes = get_nodes_array(mesh_data)
    conn = [element['conn_list'] for element in mesh_data['conn_data']]
    conn = np.array(conn) - 1  # Ajusta a conectividade para índices baseados em 0

    # Inicializa listas para armazenar |B| e as coordenadas
    diagonal_coords = []
    B_magnitudes = []

    # Calcula o módulo de |B| e filtra os elementos na diagonal do domínio (x ≈ y)
    for e, B_element in enumerate(B_domain):
        # Obtém a conectividade do elemento
        element_nodes = conn[e]

        # Para P1, usa o centroide; para P2, considera apenas os nós vértices
        if order == 1:
            element_coords = nodes[element_nodes]
        elif order == 2 or order == 3:
            element_coords = nodes[element_nodes[:3]]  # Apenas os 3 primeiros nós (vértices)

        # Calcula o centroide do elemento
        centroid = np.mean(element_coords, axis=0)  # Centroide (x, y)

        # Filtra elementos próximos da diagonal (x = y)
        if np.isclose(centroid[0], centroid[1], atol=0.05):  # Tolerância de 0.05
            diagonal_coords.append(centroid[0])  # Coordenada x do centroide (ou y, já que x = y)
            # Calcula a magnitude média de |B| para o elemento
            B_magnitudes_element = [np.linalg.norm(B) for B in B_element]
            B_magnitude_avg = np.mean(B_magnitudes_element)
            B_magnitudes.append(B_magnitude_avg)

    # Ordena os valores em função de x (ou y, pois x = y)
    diagonal_coords = np.array(diagonal_coords)
    B_magnitudes = np.array(B_magnitudes)
    sorted_indices = np.argsort(diagonal_coords)
    diagonal_coords_sorted = diagonal_coords[sorted_indices]
    B_magnitudes_sorted = B_magnitudes[sorted_indices]

    # Plotagem do gráfico
    plt.figure(figsize=(8, 6))
    plt.semilogy(diagonal_coords_sorted, B_magnitudes_sorted, label=r'|$\mathbf{B}$|')
    plt.xlabel('Diagonal coordinate (m)')
    plt.ylabel(r'|$\mathbf{B}$| (T)')
    plt.grid(True)
    plt.legend()
    plt.tight_layout()
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")